# Opdracht 2 - Convolutional Neural Networks


Het doel van deze opdracht is om image recognition te doen aan de hand van Convolutional Neural Networks (CNN). In een eerste deelopdracht ontwerp je een binaire CNN classifier voor het classificeren van foto's van lichaamscellen die al dan niet met malaria geïnfecteerd zijn. 

In een tweede deelopdracht zal je gezichtsherkenning uitvoeren via een multi-class CNN classifier.

In een derde deelopdracht train je een CNN die gezichten van niet-gezichten kan onderscheiden. Met deze classifier kan je vervolgens gezichtsdetectie uitvoeren. Daarbij is het niet de bedoeling om gezichten te herkennen, maar de locatie van al dan niet aanwezig gezichten in afbeeldingen te vinden en aan te duiden.

Na deze opdracht zal je vertrouwd zijn met:
- Implementeren van convolutional neural networks voor objectherkenning in afbeeldingen.
- Hyperparameter tuning van CNN's
- Weten hoe je transfer learning kan toepassen voor jouw eigen classificatie taak.
- Data/Image augmentation uitvoeren.
- Object detectie via een sliding window.


In [6]:
%matplotlib inline

import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
import matplotlib.image as mpimg
from skimage.io import imread, imsave, imshow
from skimage import data, color, io, filters, morphology,transform, exposure, feature, util
from scipy import ndimage
import sklearn.metrics as metrics
from sklearn.utils import class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

import re

#K.set_image_dim_ordering('tf')
plt.rcParams['image.cmap'] = 'gray'
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)

## Malaria Classification

Ontwerp en train een CNN dat is in staat is met Malaria geïnfecteerde cellen van niet-geïnfecteerde cellen te onderscheiden.

De data is te vinden in:
- './Malaria/train/infected/': afbeelding van met Malaria geïnfecteerde cellen om mee te trainen.
- './Malaria/train/uninfected/': afbeeldingen van gezonde, niet-geïnfecteerde cellen om mee te trainen.
- './Malaria/test/infected/': afbeelding van met Malaria geïnfecteerde cellen om mee te testen.
- './Malaria/test/uninfected/': afbeeldingen van gezonde, niet-geïnfecteerde cellen om mee te testen.


1. Ontwerp in eerste instantie een eigen CNN. 
- Probeer de accuracy van het CNN op de test data zo hoog mogelijk te krijgen. Dit kan bijvoorbeeld omvatten: hyperparameter tuning van de het netwerk zelf, early stopping, image augmentation of eigen bedachte (pre)-processing technieken. 
- Stel nu dat aan het CNN de vereiste wordt gesteld dat het aantal false negatives (het netwerk voorspelt niet-geïnfecteerd terwijl ze wel geïnfecteerd zijn) op de test set maximaal 1% mag bedragen. Welke aanpassingen zou je doen? Voor deze aanpassingen uit en toon aan dat het aantal false negatives tot maximum 1% is gezakt. Welke accuracy haal je nog?

2. Voer transfer learning uit. Hertrain het VGG19 netwerk voor het classificeren van de al dan niet met Malaria geïnfecteerde cellen. Vergelijk de resultaten met deze van het eigen CNN in termen van accuraatheid, f1-score, snelheid van training, de benodigde hoeveelheid training data, etc.



### Eigen CNN

In [7]:
# Inlezen van de data

# Lezen en preprocessen 

image_size = 100  # hoogte en breedte van de afbeeldingen na resizing. Pas gerust aan.
nr_train_images = 1000  # Om het aantal afbeelding in de training set te beperken. Verhoog indien jouw systeem het toelaat.
nr_test_images = 1000 # Om het aantal afbeelding in de test set in eerste instantiete beperken. 
infected_train_images = []
infected_test_images = []
uninfected_train_images = []
uninfected_test_images = []
y_infected_train = []
y_uninfected_train = []
y_infected_test = []
y_uninfected_test =[]

# read infected train_images
path = './Malaria/train/infected/'
valid_images = [".jpg",".gif",".png"]

for f in os.listdir(path)[:nr_train_images]:
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    im = imread(os.path.join(path,f)) 
    im = transform.resize(im,(image_size,image_size),mode='constant',anti_aliasing=True)
    infected_train_images.append(im)
    y_infected_train.append(1)
    
# read infected test_images

path = './Malaria/test/infected/'
valid_images = [".jpg",".gif",".png"]

for f in os.listdir(path)[:nr_test_images]:
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    im = imread(os.path.join(path,f))
    im = transform.resize(im,(image_size,image_size),mode='constant',anti_aliasing=True)
    infected_test_images.append(im)
    y_infected_test.append(1)
    

# read uninfected train_images
path = './Malaria/train/uninfected/'
valid_images = [".jpg",".gif",".png"]

for f in os.listdir(path)[:nr_train_images]:
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    im = imread(os.path.join(path,f)) 
    im = transform.resize(im,(image_size,image_size),mode='constant',anti_aliasing=True)
    uninfected_train_images.append(im)
    y_uninfected_train.append(0)


# read uninfected test_images

path = './Malaria/test/uninfected/'
valid_images = [".jpg",".gif",".png"]

for f in os.listdir(path)[:nr_test_images]:
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    im = imread(os.path.join(path,f)) 
    im = transform.resize(im,(image_size,image_size),mode='constant',anti_aliasing=True)
    uninfected_test_images.append(im)
    y_uninfected_test.append(0)
    


In [ ]:
# Toon een afbeelding

plt.imshow(uninfected_train_images[1])

In [ ]:
# Creëer te de test set en training set. Zorg ervoor dat ze gerandomiseerd zijn.




In [ ]:
# Normalisatie van de data



In [ ]:
# CNN



In [1]:
# Testen van het neuraal netwerk op de test set. Kijk naar de accuracy, recall, precision, f1-score en de ROC 



In [3]:
# Optimalisatie van het netwerk



In [ ]:
# Visualiseer enkele verkeerd geclassificeerde afbeeldingen van de ene klasse en van de andere klasse. 


In [ ]:
# Verminderen van het aantal false negatives tot max 1%.



**Schrijf hier jouw conclusies:**



### Transfer learning
Hertrain het VGG19 netwerk (of optioneel ResNet) op de malaria dataset. Vergelijk de resultaten met deze van het eigen CNN in termen van accuraatheid, f1-score, snelheid van training, de benodigde hoeveelheid training data, etc.

In [10]:
# transfer learning


#modelVGG19 = tf.keras.applications.vgg19.VGG19()
modelVGG19 = tf.keras.applications.vgg19.VGG19(include_top=False, weights='imagenet', input_shape=(100,100,3))
type(modelVGG19)


# Converteer naar een sequential model.


    
# Zet de gewichten van feature extraction layers vast.


# Voeg dense layers toe.


tensorflow.python.keras.engine.training.Model

**Schrijf hier jouw conclusies:**


## Face recognition

Train een eigen CNN dat als taak heeft een zo accuraat mogelijke gezichtsherkenning uit te voeren op een gezichtsdataset van celebrities.

Er zijn twee gezichtsdatasets beschikbaar:

'./Face_Recognition/15_Classes' bevat gezichten van 15 verschillende celbrities.
'./Face_Recognition/100_Classes' bevat gezichten van 100 verschillende celebrities.

Werk in eerste instantie met de 15_classes dataset. 


### 15 classes dataset - eigen CNN

In [8]:
# Inlezen en preprocessen van de afbeeldingen
# nr_of_images = 5000
image_size = 100


faces = []
labels = []
# read infected train_images
path = './Face_Recognition/15_Classes'
valid_images = [".jpg",".gif",".png"]

for f in os.listdir(path)[:]:
    ext = os.path.splitext(f)[1]
    if ext.lower() not in valid_images:
        continue
    im = imread(os.path.join(path,f)) 
    im = transform.resize(im,(image_size,image_size),mode='constant',anti_aliasing=True)
    faces.append(im)
    #s = re.sub('[0-9]\w+', '', f)
    s = os.path.splitext(f)[0]
    s = ''.join([i for i in s if not i.isdigit()])
    s = s.replace("_", "")
    labels.append(s)

In [ ]:
# Creëer een training set en test set. Zorg ervoor dat 30 procent van de afbeeldingen in de test set zitten.



In [ ]:
# Convolutional Neural Network



In [ ]:
# Testen van het CNN



In [4]:
# Hyperparameter tuning en image augmentation. 


**Schrijf hier jouw conclusies neer:**

### Transfer learning by means of VGG19

Hertrain een VGG19 net (of Resnet).

In [5]:
# VGG19


**Schrijf hier jouw conclusies neer:**

### 100 classes dataset

Probeer nu classificatie te doen op de 100_classes dataset. Tranfer learning hoeft niet. Vergelijk de resultaten met de performantie op de 15_classes dataset. Evalueer en verklaar de resultaten.

In [6]:
# Herkenning van 100 classes dataset



**Schrijf hier jouw conclusies neer:**

## Face detection 


Het doel is het ontwerpen van een CNN die met een hoge accuraatheid in staat is om gezichten te detecteren waarna rond het gezicht een bounding box getekend wordt. 
Bijvoorbeeld:
![alt text](./Jupyter_Images/Face_Detection.png) 

Doorloop de volgede stappen:
- Compileer een training set met gezichten en niet-gezichten. Online zijn verschillende gezichtsdatasets te vinden. Bijvoorbeeld: https://lionbridge.ai/datasets/5-million-faces-top-15-free-image-datasets-for-facial-recognition/ en http://shuoyang1213.me/WIDERFACE/
- Train een CNN (mag via transfer learning, maar hoeft niet) die met een hoge accuraatheid gezichten van niet-gezichten kan onderscheiden.
- Test deze CNN om een idee te hebben van de classifiation accuracy. 
- Indien de performantie voldoende hoog is kan je een sliding window detector implementeren. Dit betekent dat je de test-afbeelding meerdere keren afscant met een sliding window van verschillende groottes. Het CNN zal voor elke positie van het sliding window een classificatie doen van de subimage die onder het window is gelegen. Teken een bounding box wanneer de subimage een gezicht bevat.

Voor het tekenen van een bounding box kan de volgende code gebruikt worden:

```python
def rectangle_perimeter(r0, c0, width, height, shape=None, clip=False):
    rr, cc = [r0, r0 + width, r0 + width, r0], [c0, c0, c0 + height, c0 + height]
    return skimage.draw.polygon_perimeter(rr, cc, shape=shape, clip=clip)

# drawing the bounding box:
rr, cc = rectangle_perimeter(y, x, w, w)
image_detected[rr,cc] =255

```

Extra uitbreiding:

Probleem: omdat we de afbeelding meerdere keren afscannen, telkens met windows van verschillende grootte is het goed mogelijk dat er meerdere bounding boxes worden geteked rond hetzelfde gezicht. Zoek naar een manier om meerdere bounding boxes die bij hetzelfde gezicht horen samen te voegen tot 1 bounding box.
Een veel gebruikte techniek is Non-Maximum Suppression: https://www.pyimagesearch.com/2014/11/17/non-maximum-suppression-object-detection-python/ en https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/.